In [2]:
# Text File Data Extraction to Pandas DataFrame

import pandas as pd
import re
import os
from pathlib import Path

# Define extraction patterns and column mappings
EXTRACTION_PATTERNS = [
    {
        'pattern': r"Target Class: \['([^']+)'\]",
        'column': 'Tumor type',
        'description': 'Extract tumor type from Target Class'
    },
    {
        'pattern': r"Best parameters: \{[^}]*'C': ([^,}]+)",
        'column': 'C',
        'description': 'Extract C parameter from Best parameters',
        'convert_type': float
    },
    {
        'pattern': r"Best parameters: \{[^}]*'l1_ratio': ([^,}]+)",
        'column': 'l1_ratio',
        'description': 'Extract l1_ratio from Best parameters',
        'convert_type': float
    },
    {
        'pattern': r"Number of significant proteins: (\d+)",
        'column': 'Proteins selected',
        'description': 'Extract number of significant proteins',
        'convert_type': int
    },
    {
        'pattern': r"Selected hyperparameter C: ([^\s;]+)",
        'column': 'Ridge HP',
        'description': 'Extract Ridge hyperparameter C',
        'convert_type': float
    },
    {
        'pattern': r"Train MCC Score: ([\d\.]+ ± [\d\.]+)",
        'column': 'MCC Train',
        'description': 'Extract Train MCC Score with standard deviation'
    },
    {
        'pattern': r"MCC test: ([\d\.]+)",
        'column': 'MCC Test',
        'description': 'Extract MCC test score',
        'convert_type': float
    },
    {
        'pattern': r"Confusion Matrix:\s+TN \| FP\s+\[\[(\d+)\s+(\d+)\]\s+\[\s*(\d+)\s+(\d+)\]\]\s+FN \| TP",
        'column': 'TN',
        'description': 'Extract True Negatives from confusion matrix',
        'convert_type': int,
        'group': 1  # First capture group
    },
    {
        'pattern': r"Confusion Matrix:\s+TN \| FP\s+\[\[(\d+)\s+(\d+)\]\s+\[\s*(\d+)\s+(\d+)\]\]\s+FN \| TP",
        'column': 'FP',
        'description': 'Extract False Positives from confusion matrix',
        'convert_type': int,
        'group': 2  # Second capture group
    },
    {
        'pattern': r"Confusion Matrix:\s+TN \| FP\s+\[\[(\d+)\s+(\d+)\]\s+\[\s*(\d+)\s+(\d+)\]\]\s+FN \| TP",
        'column': 'FN',
        'description': 'Extract False Negatives from confusion matrix',
        'convert_type': int,
        'group': 3  # Third capture group
    },
    {
        'pattern': r"Confusion Matrix:\s+TN \| FP\s+\[\[(\d+)\s+(\d+)\]\s+\[\s*(\d+)\s+(\d+)\]\]\s+FN \| TP",
        'column': 'TP',
        'description': 'Extract True Positives from confusion matrix',
        'convert_type': int,
        'group': 4  # Fourth capture group
    }
]

def extract_data_from_text(text, patterns):
    """
    Extract data from text using defined patterns
    
    Args:
        text (str): Input text to process
        patterns (list): List of pattern dictionaries
    
    Returns:
        dict: Extracted data with column names as keys
    """
    extracted_data = {}
    
    for pattern_info in patterns:
        pattern = pattern_info['pattern']
        column = pattern_info['column']
        convert_type = pattern_info.get('convert_type', str)
        
        # In the extract_data_from_text function, modify the matching section:
        match = re.search(pattern, text)
        if match:
            group_num = pattern_info.get('group', 1)  # Default to group 1
            value = match.group(group_num).strip()
            
            # Convert type if specified
            if convert_type != str:
                try:
                    value = convert_type(value)
                except (ValueError, TypeError):
                    print(f"Warning: Could not convert '{value}' to {convert_type} for column '{column}'")
            
            extracted_data[column] = value
        else:
            extracted_data[column] = None
            print(f"Warning: Pattern for '{column}' not found in text")
    
    return extracted_data

def process_single_file(file_path, patterns):
    """
    Process a single text file and extract data
    
    Args:
        file_path (str): Path to the text file
        patterns (list): List of extraction patterns
    
    Returns:
        dict: Extracted data from the file
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
        
        print(f"Processing file: {file_path}")
        extracted_data = extract_data_from_text(text, patterns)
        
        # Add filename for reference
        extracted_data['Source_File'] = os.path.basename(file_path)
        
        return extracted_data
    
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found")
        return None
    except Exception as e:
        print(f"Error processing file '{file_path}': {str(e)}")
        return None

def process_multiple_files(file_paths, patterns):
    """
    Process multiple text files and combine into DataFrame
    
    Args:
        file_paths (list): List of file paths to process
        patterns (list): List of extraction patterns
    
    Returns:
        pandas.DataFrame: Combined data from all files
    """
    all_data = []
    
    for file_path in file_paths:
        file_data = process_single_file(file_path, patterns)
        if file_data:
            all_data.append(file_data)
    
    if all_data:
        df = pd.DataFrame(all_data)
        return df
    else:
        print("No data extracted from any files")
        return pd.DataFrame()

# Example usage:

# Method 1: Process files from a directory
def process_files_from_directory(directory_path, file_extension='.txt'):
    """
    Process all files with specified extension from a directory
    """
    directory = Path(directory_path)
    if not directory.exists():
        print(f"Directory '{directory_path}' does not exist")
        return pd.DataFrame()
    
    file_paths = list(directory.glob(f'*{file_extension}'))
    if not file_paths:
        print(f"No {file_extension} files found in '{directory_path}'")
        return pd.DataFrame()
    
    print(f"Found {len(file_paths)} {file_extension} files")
    return process_multiple_files(file_paths, EXTRACTION_PATTERNS)

# Method 2: Process specific files
def process_specific_files(file_list):
    """
    Process a specific list of files
    """
    return process_multiple_files(file_list, EXTRACTION_PATTERNS)


In [3]:

# Example: Process all txt files in current directory
print("=== Processing files from current directory ===")
df_from_directory = process_files_from_directory('.', '.txt')

if not df_from_directory.empty:
    print("\nDataFrame created successfully!")
    print(f"Shape: {df_from_directory.shape}")
    print("\nColumn names:")
    print(df_from_directory.columns.tolist())
    print("\nFirst few rows:")
    print(df_from_directory.head())
    
    # Display data types
    print("\nData types:")
    print(df_from_directory.dtypes)
    
    # Check for missing values
    print("\nMissing values:")
    print(df_from_directory.isnull().sum())
else:
    print("No DataFrame created - no files processed successfully")

# Example: Process specific files (uncomment and modify as needed)
"""
specific_files = [
    'file1.txt',
    'file2.txt', 
    'file3.txt'
]
df_specific = process_specific_files(specific_files)
"""

# Save to CSV (optional)
if not df_from_directory.empty:
    output_filename = 'extracted_data.csv'
    df_from_directory.to_csv(output_filename, index=False)
    print(f"\nData saved to '{output_filename}'")

# Display final DataFrame
print("\n=== Final DataFrame ===")
if not df_from_directory.empty:
    display(df_from_directory)
else:
    print("No data to display")

# Function to test patterns on sample text
def test_patterns_on_sample():
    """
    Test the extraction patterns on sample text
    """
    sample_text = """
    Target Class: ['ACYC']
    Best parameters: {'C': 2, 'l1_ratio': 0.5}
    Number of significant proteins: 20
    Selected hyperparameter C: 10
    Train MCC Score: 0.7104 ± 0.0940
    MCC test: 0.591327405378887
    """
    
    print("=== Testing patterns on sample text ===")
    print("Sample text:")
    print(sample_text)
    print("\nExtracted data:")
    
    extracted = extract_data_from_text(sample_text, EXTRACTION_PATTERNS)
    for key, value in extracted.items():
        print(f"{key}: {value} (type: {type(value).__name__})")

# Uncomment to test patterns
# test_patterns_on_sample()

=== Processing files from current directory ===
Found 14 .txt files
Processing file: classifier_output_20250828_043105.txt
Processing file: classifier_output_20250825_000943.txt
Processing file: classifier_output_20250825_091516.txt
Processing file: classifier_output_20250825_035529.txt
Processing file: classifier_output_20250827_202942.txt
Processing file: classifier_output_20250825_153817.txt
Processing file: ttc_pip_requirements.txt
Processing file: classifier_output_20250824_204240.txt
Processing file: classifier_output_20250827_093711.txt
Processing file: classifier_output_20250828_001240.txt
Processing file: classifier_output_20250827_145047.txt
Processing file: classifier_output_20250826_160345.txt
Processing file: classifier_output_20250824_172007.txt
Processing file: classifier_output_20250828_093122.txt

DataFrame created successfully!
Shape: (14, 12)

Column names:
['Tumor type', 'C', 'l1_ratio', 'Proteins selected', 'Ridge HP', 'MCC Train', 'MCC Test', 'TN', 'FP', 'FN', 'TP

,Tumor type,C,l1_ratio,Proteins selected,Ridge HP,MCC Train,MCC Test,TN,FP,FN,TP,Source_File
0,GIST,3.0,0.3,116.0,1.0,0.6850 ± 0.2275,0.893262,383.0,1.0,0.0,4.0,classifier_output_20250828_043105.txt
1,None,1.0,0.3,67.0,10.0,0.8279 ± 0.1471,0.772582,383.0,0.0,2.0,3.0,classifier_output_20250825_000943.txt
2,THYC,5.0,0.3,427.0,0.1,0.7625 ± 0.2148,0.893262,383.0,1.0,0.0,4.0,classifier_output_20250825_091516.txt
3,THYM,10.0,0.3,528.0,0.1,0.9664 ± 0.0887,1.000000,463.0,0.0,0.0,3.0,classifier_output_20250825_035529.txt
4,CHS,1.0,0.3,56.0,10.0,0.5090 ± 0.2631,0.000000,384.0,0.0,4.0,0.0,classifier_output_20250827_202942.txt
5,MFH,5.0,0.5,63.0,10.0,0.3149 ± 0.0856,0.449657,449.0,4.0,8.0,5.0,classifier_output_20250825_153817.txt
6,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,ttc_pip_requirements.txt
7,BA,1.0,0.5,27.0,10.0,0.5730 ± 0.2222,0.329014,461.0,2.0,2.0,1.0,classifier_output_20250824_204240.txt
8,THYM,3.0,0.3,203.0,0.1,0.7977 ± 0.2460,1.000000,463.0,0.0,0.0,3.0,classifier_output_20250827_093711.txt
9,DFSP,1.5,0.3,127.0,0.1,1.0000 ± 0.0000,0.865090,462.0,1.0,0.0,3.0,classifier_output_20250828_001240.txt
